---
title: "Depth"
---

> Explore the effects of changing the canal depth (test).

Text goes here


In [ ]:
#| echo: false
#| warning: false

import glob
import json
from datetime import datetime
import plotly.graph_objects as go
import plotly.io as pio

# List files of interest
variable = "draught"
files = glob.glob(f'./data/rc_9_including_{variable}*.json')

# Create a Plotly figure, colour list
fig = go.Figure()
colours = ["#4daeff", "#b64dff", "#ff5d4d", "#ff7000"]

# Iterate and load
for f, c in zip(files, colours):
    with open(f, 'r') as file:
        data = json.load(file)

    # Extract lists of values to plot, converting to celcius
    modelled_water = [x['surface_water_k'] - 273.15 for x in data.values()]

    # List of dict keys (unix), convert to datetime format
    modelled_dt = list(data.keys())
    modelled_datetime = [datetime.utcfromtimestamp(int(x)) for x in modelled_dt]

    # Add trace
    if int(f[-6]) == 0:
        fig.add_trace(go.Scatter(x=modelled_datetime, y=modelled_water, mode='lines', 
        line_color = c, opacity=0.5, name=f"+{f[-6]}°C"))

    # Else, hide to start
    else:
        fig.add_trace(go.Scatter(x=modelled_datetime, y=modelled_water, mode='lines', 
        line_color = c, opacity=0.5, visible="legendonly", name=f"+{f[-6]}°C"))


# Axis labels
fig.update_layout(
    xaxis_title="Date", yaxis_title="Water temperature (°C)", 
    legend_title="Air temperature",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))

# Save the figure as an HTML file
#fig.write_html("./images/depth_water_temperature.html")

fig.show()

<iframe src="./images/depth_water_temperature.html" width="100%" height="600px"></iframe>


In [ ]:
#| echo: false
#| warning: false

import glob
import json
from datetime import datetime
import plotly.graph_objects as go
import plotly.io as pio
from pandas import to_datetime
from datetime import timedelta

# Load summary file of interest
variable = "draught"
with open(f'./data/rc_9-{variable}-summary.json', 'r') as file:
        data = json.load(file)

# Localise niave datetime to aware datetime (UK) and then convert to UTC
local_datetime = to_datetime("2022-01-01 10:00:00")\
.tz_localize('Europe/London')\
.tz_convert('UTC')

# Create a Plotly figure, colour list
fig = go.Figure()
colours = ["#4daeff", "#b64dff", "#ff5d4d", "#ff7000"]

# Extract and sort keys
key_list = data.keys()
sorted_keys = sorted(key_list, key=lambda data: float(data[30:][:-5]))

# Iterate and extract data
for k, c in zip(sorted_keys, colours):
  value = data[k]

  # X-axis position
  change = float(k[30:][:-5])

  # Extract values
  median = value['day values']['median']
  ymin = abs(median - value['day values']['q1'])
  ymax = abs(median - value['day values']['q3'])

  # Add to plot
  if change == 1:
    fig.add_scatter(x = [change], 
    y = [median],
    name = f"×{change}",
    mode='markers',
    marker=dict(color=c),
    error_y = dict(type = 'data', color = c,
    symmetric = False,
    array = [ymax],
    arrayminus = [ymin]))

  # Hide to start
  else:
    fig.add_scatter(x = [change], 
    y = [median],
    name = f"+{change} m",
    mode='markers',
    marker=dict(color=c),
    visible='legendonly',
    error_y = dict(type = 'data', color = c,
    symmetric = False,
    array = [ymax],
    arrayminus = [ymin]))


# Axis labels
fig.update_layout(
    xaxis_title="Increase in canal depth (m)", yaxis_title="Air temperature effect (°C)", 
    legend=dict(orientation="h",
    yanchor="bottom", y=1.02,
    xanchor="left", x=0, 
    title = "Canal depth"
))

# Save the figure as an HTML file
#fig.write_html("./images/draught_effects.html")

fig.show()

<iframe src="./images/draught_effects.html" width="100%" height="600px"></iframe>

> How does canal area influence the magnitude of the effect of the surrounding air?